In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [16]:
# Load Titanic dataset
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Define features and target
X = df.drop(columns=['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'])
y = df['Survived']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [17]:
def feature_engineering(X):
    X = X.copy()
    # Create 'FamilySize'
    X['FamilySize'] = X['SibSp'] + X['Parch']
    # Create 'IsAlone'
    X['IsAlone'] = (X['FamilySize'] == 0).astype(int)
    return X

# Wrap the function in a FunctionTransformer
feature_engineering_transformer = FunctionTransformer(feature_engineering)

In [18]:
# Define columns
numerical_features = ['Age', 'Fare', 'FamilySize', 'SibSp', 'Parch']
categorical_features = ['Sex', 'Embarked', 'Pclass']

# Preprocessing for numerical features
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Combine pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='drop'  # Drop unused columns (e.g., 'IsAlone' is already derived)
)

In [19]:
pipeline = Pipeline([
    # Step 1: Feature engineering
    ('feature_engineering', feature_engineering_transformer),
    
    # Step 2: Preprocessing (imputation, encoding, scaling)
    ('preprocessor', preprocessor),
    
    # Step 3: Classifier
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [20]:
# Train the pipeline
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.82


In [21]:
# Example: Predict survival for a new passenger
new_passenger = pd.DataFrame({
    'Pclass': [2],
    'Sex': ['female'],
    'Age': [29],
    'SibSp': [0],
    'Parch': [0],
    'Fare': [8.05],
    'Embarked': ['S']
})

prediction = pipeline.predict(new_passenger)
print("Survived" if prediction[0] else "Did not survive")

Survived
